In [1]:
import gc
import os
from typing import Tuple

import keras
import mat73
import matplotlib.pyplot as plt
import numpy as np
import scipy
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
from tqdm import tqdm

from nowcasting.unet import res1, res2

2022-11-19 19:13:06.427444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 19:13:06.640929: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-19 19:13:07.550914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-19 19:13:07.550982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or direc

In [2]:
def sliding_window_expansion(
        X: np.ndarray,
        y: np.ndarray,
        input_window_size: int = 3,
        target_window_size: int = 1,
        target_offset: int = 1,
        step: int = 2,
        sample_ratio: int = 1) -> Tuple[np.ndarray, np.ndarray]:
    """
    sliding_window_expansion Generates input and target pairs from time series like dataset

    Parameters
    ----------
    X : np.ndarray
        Input timeseries array
    y : np.ndarray
        Target timeseries array
    input_window_size : int, optional
        Number of timesteps in each input, by default 3
    target_window_size : int, optional
        Number of timesteps in each target, by default 1
    target_offset : int, optional
        Number of timesteps the start of each target should be from the end of each input, by default 1
    step : int, optional
        Number of timesteps to use to determine the start of the next input and target pair, by default 2
    sample_ratio : int, optional
        Percentage of results to return, by default 1

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        Input and target timeseries pairs
    """
    max_idx = X.shape[
        0] - input_window_size - target_window_size - target_offset
    x_idx = (np.expand_dims(np.arange(input_window_size), 0) +
             np.expand_dims(np.arange(max_idx, step=step), 0).T)
    y_idx = (np.expand_dims(np.arange(target_window_size), 0) +
             np.expand_dims(np.arange(max_idx, step=step),
                            0).T) + input_window_size + target_offset

    if sample_ratio < 1:
        sample_mask = (np.random.random(size=(len(x_idx))) <= sample_ratio)
        x_idx = x_idx[sample_mask]
        y_idx = y_idx[sample_mask]

    X_new = X[x_idx]
    y_new = y[y_idx]

    return X_new, y_new

In [3]:
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
tf.config.optimizer.set_experimental_options({"layout_optimizer": False})

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

print(tf.config.list_physical_devices('GPU'))

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080 Ti, compute capability 7.5
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
mat_path = "../data/full_sample"
mat_files = [f"{mat_path}/{x}" for x in os.listdir(mat_path)]
mat_files.sort()

In [5]:
Xs = []
ys = []
for mat_file in tqdm(mat_files):
    mat = scipy.io.loadmat(mat_file)
    mat_shape = mat["X"]["imerg"][0][0].shape
    Xs.append(
        np.array([mat["X"][x][0][0]
                  for x in ["imerg", "gfs_v", "gfs_tpw"]]).reshape(
                      (mat_shape[0], mat_shape[1], 3)))
    ys.append(mat["X"]["gfs_pr"][0][0].reshape(
        (mat_shape[0], mat_shape[1], 1)))

Xs = np.array(Xs)
ys = np.array(ys)


100%|██████████| 480/480 [00:06<00:00, 74.54it/s]


In [6]:
print(Xs.shape)
print(ys.shape)

(480, 256, 620, 3)
(480, 256, 620, 1)


In [7]:
X, y = sliding_window_expansion(Xs,
                                ys,
                                input_window_size=12,
                                target_window_size=8,
                                target_offset=0,
                                step=8,
                                sample_ratio=1)


In [8]:
# X_new = []
# y_new = []
# for i in range(X.shape[0]):
#     X_new.append(X[i, :, :, :256, :])
#     y_new.append(y[i, :, :, :256, :])

# X = np.array(X_new)
# y = np.array(y_new)

In [9]:
print(X.shape)
print(y.shape)

(58, 12, 256, 620, 3)
(58, 8, 256, 620, 1)


In [10]:
train_cutoff = int(X.shape[0] * 0.9)
X_train = X[:train_cutoff]
y_train = y[:train_cutoff]
X_test = X[train_cutoff:]
y_test = y[train_cutoff:]

In [11]:
del Xs, ys, mat
gc.collect()

0

In [12]:
model = res2((12, 256, 620, 3), num_filters_base=8, dropout_rate=0.2)
model.summary()
# with open("modelsummary.txt", "w") as f:
#     model.summary(print_fn=lambda x: f.write(x + "\n"))

2022-11-19 19:13:17.960259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 19:13:18.694855: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-11-19 19:13:18.694883: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2022-11-19 19:13:18.695017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 256, 62  0           []                               
                                0, 3)]                                                            
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 256, 620  12         ['input_1[0][0]']                
 alization)                     , 3)                                                              
                                                                                                  
 zero_padding3d (ZeroPadding3D)  (None, 12, 256, 624  0          ['batch_normalization[0][0]']    
                                , 3)                                                          

In [13]:
#%%

model.compile(loss="mean_absolute_error",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["mae", "mse"])
#mean_absolute_error
checkpoint_filepath = "script_n1.h5"
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint(filepath=checkpoint_filepath,
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True)
]

#%%
results = model.fit(X_train,
                    y_train,
                    batch_size=2,
                    epochs=50,
                    callbacks=callbacks,
                    verbose=1,
                    validation_data=(X_test, y_test))


2022-11-19 19:13:22.203935: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1188495360 exceeds 10% of free system memory.
2022-11-19 19:13:23.614442: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1188495360 exceeds 10% of free system memory.


Epoch 1/50


2022-11-19 19:14:07.756606: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-11-19 19:14:08.640605: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-19 19:14:09.360494: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 67895296 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 41877504/11552227328
2022-11-19 19:14:09.360551: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                     10089332736
InUse:                     10517751812
MaxInUse:                  10517751812
NumAllocs:                       13706
MaxAllocSize:               1188495360
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-11-19 19:14:09.360894: E tensorflow/core

ResourceExhaustedError: Graph execution error:

Detected at node 'model/concatenate_1/concat' defined at (most recent call last):
    File "/home/pygmy_lord/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/pygmy_lord/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/pygmy_lord/repos/Nowcasting/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/pygmy_lord/repos/Nowcasting/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/pygmy_lord/repos/Nowcasting/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/pygmy_lord/repos/Nowcasting/.venv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loo

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
y_hat.shape

In [ ]:
y_test.shape

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 50))

# ax.imshow(np.abs(y_hat[0, 0, :, :] - y_test[0, 1, :, :]))
ax.imshow(y_test[0, 0, :, :, :])

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 50))

# ax.imshow(np.abs(y_hat[0, 0, :, :] - y_test[0, 1, :, :]))
ax.imshow(y_hat[0, 0, :, :])

fig.tight_layout()